# Расчет цен опционов call и put на исторических данных MOEX

### Импорт необходимых библиотек, загрузка и предобработка данных

In [20]:
import numpy as np
import pandas as pd

In [21]:
option_stat = pd.read_csv(
    './data/moex-GAZP-241225-marg-optionsdesk.csv',
    sep=',',
    encoding='cp1251'
)

In [22]:
option_stat.head()

,CALL: Открыт.позиций,"CALL: Последняя сделка, Значение","CALL: Последняя сделка, Дата","CALL: Последняя сделка, Изменение",CALL: ПОКУПКА,CALL: ПРОДАЖА,CALL: Расчетная цена,CALL: Теоретическая цена,СТРАЙК,IV,PUT: Теоретическая цена,PUT: Расчетная цена,PUT: ПОКУПКА,PUT: ПРОДАЖА,"PUT: Последняя сделка, Значение","PUT: Последняя сделка, Дата","PUT: Последняя сделка, Изменение",PUT: Открыт. позиций,Unnamed: 18
0,NaN,NaN,NaN,NaN,13.58,493.20,24.64,24.25,100.0,126.60,0.01,0.01,NaN,0.70,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,13.58,393.40,19.64,19.26,105.0,111.63,0.01,0.01,NaN,0.70,NaN,NaN,NaN,NaN,NaN
2,17600.0,14.39,23.12.25 10:54:54,-9.15,13.90,14.53,14.65,14.27,110.0,87.85,0.02,0.01,NaN,0.23,NaN,NaN,NaN,1100.0,NaN
3,84000.0,NaN,NaN,NaN,9.05,9.52,9.65,9.28,115.0,60.72,0.02,0.01,NaN,0.24,NaN,NaN,NaN,1942.0,NaN
4,87636.0,4.55,23.12.25 13:22:30,-9.00,4.04,4.55,4.84,4.34,120.0,39.85,0.08,0.19,0.01,0.23,0.08,23.12.25 13:26:34,-65.22,38356.0,NaN


In [23]:
option_stat = option_stat[['CALL: Последняя сделка, Значение', 'PUT: Последняя сделка, Значение', 'СТРАЙК', 'IV']]

In [24]:
option_stat = option_stat.rename(columns={'CALL: Последняя сделка, Значение': 'call_price', 
                                          'PUT: Последняя сделка, Значение': 'put_price',
                                          'СТРАЙК': 'strike_price',
                                          'IV': 'implied_volatility'})

In [25]:
option_stat.head()

,call_price,put_price,strike_price,implied_volatility
0,NaN,NaN,100.0,126.60
1,NaN,NaN,105.0,111.63
2,14.39,NaN,110.0,87.85
3,NaN,NaN,115.0,60.72
4,4.55,0.08,120.0,39.85


In [26]:
option_stat = option_stat.dropna(subset=['call_price', 'put_price'])

In [27]:
option_stat

,call_price,put_price,strike_price,implied_volatility
4,4.55,0.08,120.0,39.85
5,0.44,1.58,125.0,32.65
6,0.01,5.78,130.0,41.26
7,0.10,11.10,135.0,60.48


### Импортируем класс для предсказания цены опциона при помощи уравнения Блэка-Шоулза

In [28]:
from blackscholes import BlackScholes

In [29]:
bs = BlackScholes()

In [30]:
S = 124.21          # цена базового актива (GAZP)
T = 1 / 365         # время до экспирации в годах (дата экспирации - 24.12.2025)
r = 0.10            # безрисковая ставка (принята 10% годовых)

In [31]:
for idx, row in option_stat.iterrows():
    K = row['strike_price']
    sigma = row['implied_volatility'] / 100
    call_price = bs.call_price(S, K, T, r, sigma)
    put_price = bs.put_price(S, K, T, r, sigma)
    option_stat.at[idx, 'call_price_theoretical'] = call_price
    option_stat.at[idx, 'put_price_theoretical'] = put_price

In [32]:
option_stat.head()

,call_price,put_price,strike_price,implied_volatility,call_price_theoretical,put_price_theoretical
4,4.55,0.08,120.0,39.85,4.293400,0.050528
5,0.44,1.58,125.0,32.65,0.524484,1.280242
6,0.01,5.78,130.0,41.26,0.017898,5.772286
7,0.10,11.10,135.0,60.48,0.005579,10.758598


### Вычислим среднюю абсолютную ошибку предсказания цены опциона

In [40]:
put_price_error = np.mean(abs(option_stat['put_price_theoretical'] - option_stat['put_price']))
print(f'Ошибка MAE для опционов `put`: {put_price_error}')

Ошибка MAE для опционов `put`: 0.16958649524021752


In [42]:
call_price_error = np.mean(abs(option_stat['call_price_theoretical'] - option_stat['call_price']))
print(f'Ошибка MAE для опционов `call`: {call_price_error}')

Ошибка MAE для опционов `call`: 0.11085061332761298
